In [2]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("../data/raw/bitcoin.csv")#.drop(columns=["Unnamed: 0"])

data.info()

<class 'pandas.DataFrame'>
RangeIndex: 8729 entries, 0 to 8728
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       8729 non-null   str    
 1   Adj Close  8729 non-null   float64
 2   Close      8729 non-null   float64
 3   High       8729 non-null   float64
 4   Low        8729 non-null   float64
 5   Open       8729 non-null   float64
 6   Volume     8729 non-null   int64  
dtypes: float64(5), int64(1), str(1)
memory usage: 477.5 KB


In [3]:
data.head()

,date,Adj Close,Close,High,Low,Open,Volume
0,2025-02-18 00:00:00+00:00,95595.156250,95595.156250,96014.375000,95567.773438,95774.218750,0
1,2025-02-18 01:00:00+00:00,96265.679688,96265.679688,96265.679688,95613.632812,95613.632812,0
2,2025-02-18 02:00:00+00:00,96054.390625,96054.390625,96311.937500,96037.515625,96261.156250,685152256
3,2025-02-18 03:00:00+00:00,96192.218750,96192.218750,96284.421875,96046.226562,96046.226562,0
4,2025-02-18 04:00:00+00:00,96056.078125,96056.078125,96287.539062,95987.539062,96186.234375,538640384


In [4]:

print(data.describe().loc["25%", "Open"])
data.describe()

88003.203125


,Adj Close,Close,High,Low,Open,Volume
count,8729.000000,8729.000000,8729.000000,8729.000000,8729.000000,8.729000e+03
mean,99585.989559,99585.989559,99846.027406,99322.804489,99589.440936,6.326736e+08
std,13708.295632,13708.295632,13682.088436,13728.195740,13704.815325,1.620208e+09
min,62978.683594,62978.683594,63744.527344,60074.804688,62801.273438,0.000000e+00
25%,87993.718750,87993.718750,88267.328125,87766.820312,88003.203125,0.000000e+00
50%,102453.664062,102453.664062,102781.335938,102198.445312,102464.109375,0.000000e+00
75%,111111.820312,111111.820312,111350.531250,110866.500000,111118.609375,7.706378e+08
max,126183.226562,126183.226562,126183.226562,125336.835938,126069.226562,4.071015e+10


In [5]:
print((data["Close"] == data["Adj Close"]).all())

True


In [6]:
datos = data.copy()

def discretizacion(data=datos, columna="close", estrategia="cuantiles"):
    if estrategia == "cuantiles":
        cuantiles = [0]
        for i in ["25%", "50%", "75%"]:
            cuantiles.append(data.describe().loc[i, columna])
        cuantiles.append(data.describe().loc["max", columna])

        etiquetas = ["g1", "g2", "g3", "g4"]
        data[f"{columna}_discreta"] = pd.cut(
            data[columna],
            bins=cuantiles,
            labels=etiquetas,
            include_lowest=True
        )

discretizacion(columna="Open")
discretizacion(columna="High")
discretizacion(columna="Low")
discretizacion(columna="Close")

datos.info()

<class 'pandas.DataFrame'>
RangeIndex: 8729 entries, 0 to 8728
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   date            8729 non-null   str     
 1   Adj Close       8729 non-null   float64 
 2   Close           8729 non-null   float64 
 3   High            8729 non-null   float64 
 4   Low             8729 non-null   float64 
 5   Open            8729 non-null   float64 
 6   Volume          8729 non-null   int64   
 7   Open_discreta   8729 non-null   category
 8   High_discreta   8729 non-null   category
 9   Low_discreta    8729 non-null   category
 10  Close_discreta  8729 non-null   category
dtypes: category(4), float64(5), int64(1), str(1)
memory usage: 511.7 KB


In [7]:
def matrices_transicion(data=datos, col_discreta="Close_discreta"):
    estados = data[col_discreta].unique().tolist()
    matriz = pd.DataFrame(0, index=estados, columns=estados, dtype=float)
    for i in range(len(data) - 1):
        estado_actual = data.loc[i, col_discreta]
        estado_siguiente = data.loc[i + 1, col_discreta]
        matriz.loc[estado_actual, estado_siguiente] += 1
        
    matriz = matriz.div(matriz.sum(axis=1), axis=0).fillna(0)
    
    return matriz

trans_open = matrices_transicion(col_discreta="Open_discreta")
trans_high = matrices_transicion(col_discreta="High_discreta")
trans_low = matrices_transicion(col_discreta="Low_discreta")
trans_close = matrices_transicion(col_discreta="Close_discreta")

print(trans_open)
print()
print(trans_high)
print()
print(trans_low)
print()
print(trans_close)
print()

          g2        g1        g3        g4
g2  0.971127  0.019707  0.009166  0.000000
g1  0.019248  0.980752  0.000000  0.000000
g3  0.009166  0.000000  0.968836  0.021998
g4  0.000000  0.000000  0.021998  0.978002

          g2        g1        g3        g4
g2  0.970669  0.021082  0.008249  0.000000
g1  0.020623  0.979377  0.000000  0.000000
g3  0.008249  0.000000  0.972502  0.019248
g4  0.000000  0.000000  0.019248  0.980752

          g2        g1        g3        g4
g2  0.971127  0.020623  0.008249  0.000000
g1  0.020165  0.979835  0.000000  0.000000
g3  0.008249  0.000000  0.974335  0.017415
g4  0.000000  0.000000  0.017415  0.982585

          g2        g1        g3        g4
g2  0.970211  0.018790  0.010999  0.000000
g1  0.018332  0.981668  0.000000  0.000000
g3  0.010999  0.000000  0.966086  0.022915
g4  0.000000  0.000000  0.022915  0.977085

